# Test for a prism data

This code calculates the amplitude of anomalous field trough an equivalent layer for a total field anomaly produced by rectangular prisms.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division
from fatiando import mesher, gridder, utils
from fatiando.gravmag import sphere
from fatiando.mesher import Sphere
from fatiando.vis import mpl
from fatiando.constants import CM, T2NT
from matplotlib import colors, ticker, cm
import mag_sphere_functions as mfun
% matplotlib inline

### Rectangular prisms

In [ ]:
inc, dec = -60., 50. # inclination and declination of regional field
incs, decs = 70., -50.      # wrong direction or remanent

In [ ]:
model = [
    mesher.Prism(-4000, -3000, -4000, -3000, 0, 2000,
                 {'magnetization': utils.ang2vec(1, inc, dec)}),
    mesher.Prism(-1000, 1000, -1000, 1000, 0, 2000,
                 {'magnetization': utils.ang2vec(1, inc, dec)}),
    # This prism will have magnetization in a different direction
    mesher.Prism(2000, 4000, 3000, 4000, 0, 2000,
                 {'magnetization': utils.ang2vec(3, -10, 45)})]

### Observation points

### Total field anomaly from rectangular prisms